In [1]:
# !pip install lightning
# !pip install protobuf
# !pip install transformers
# !pip install sentencepiece

In [2]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from transformers import DebertaTokenizer, DebertaForSequenceClassification,AutoTokenizer

import pandas as pd
from scipy.stats import spearmanr, pearsonr
from sklearn.model_selection import train_test_split
import os
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

/Users/lemarx/anaconda3/envs/str23/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class SentenceSimilarityModel(pl.LightningModule):
    def __init__(self, learning_rate=2e-5):
        super(SentenceSimilarityModel, self).__init__()
        self.deberta = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-v3-small',ignore_mismatched_sizes=True,num_labels=1)

        self.custom_layer = torch.nn.Linear(in_features=self.deberta.config.hidden_size, out_features=1)
        #Freeze parameters of the DeBERTa model
        for param in self.deberta.parameters():
            param.requires_grad = False
        self.learning_rate = learning_rate

    def forward(self, input_ids,attention_mask):
        outputs = self.deberta(input_ids, attention_mask=attention_mask)

        hidden_state = outputs.hidden_states[-1]

        logits = self.custom_layer(hidden_state[:, 0, :])
        return torch.sigmoid(logits)

    def training_step(self, batch, batch_idx):
        input_ids, attention_mask, score = batch
        logits = self(input_ids,attention_mask)

        # Assuming a binary classification task
        loss = F.mse_loss(logits, score.float())
        spearman = spearmanr(logits.detach().cpu().numpy(),score.detach().cpu().numpy()).statistic

        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log("spearman", spearman, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, attention_mask, score = batch
        logits = self(input_ids, attention_mask)

        # Assuming a binary classification task
        loss = F.mse_loss(logits, score.float())

        spearman = spearmanr(logits.detach().cpu().numpy(),score.detach().cpu().numpy()).statistic

        self.log("val_loss", loss,on_epoch = True, prog_bar = True)
        self.log("spearman",spearman,on_epoch = True, prog_bar = True)


    def configure_optimizers(self):
        return torch.optim.AdamW(self.custom_layer.parameters(), lr=self.learning_rate)


In [4]:
class str_dataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe
        self.tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        # Extract the features and target from the DataFrame
        # Adjust this based on your DataFrame structure
        features = self.dataframe['input'].loc[idx]
        token = self.tokenizer(features, return_tensors='pt', truncation='longest_first', padding='max_length',max_length=265)

        input_ids = token['input_ids'].squeeze()
        attention_mask = token['attention_mask'].squeeze()

        score = self.dataframe['Score'].loc[idx]

        score = torch.tensor(score, dtype=torch.float32).unsqueeze(dim=0)  # Adjust the dtype as needed

        return input_ids, attention_mask, score

In [5]:
class STR_DataModule(pl.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, batch_size=32):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.batch_size = batch_size

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size,num_workers = 2,persistent_workers=True, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size,num_workers = 2,persistent_workers=True)


In [6]:
train_data = pd.read_csv('/Users/lemarx/Documents/01_projects/SentencesRelatedness24/data/raw/eng_train.csv')
train_data.head()

,PairID,Text,Score
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0
3,ENG-train-0003,If he is good looking and has a good personali...,1.0
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0


In [7]:
sep = '[SEP]'
test = train_data['Text'].loc[0].replace('\n',sep)
test

'It that happens, just pull the plug.[SEP]if that ever happens, just pull the plug.'

In [8]:
train_data['input'] = train_data.apply(lambda row : row['Text'].replace('\n',sep),axis = 1)
train_data.head()

,PairID,Text,Score,input
0,ENG-train-0000,"It that happens, just pull the plug.\nif that ...",1.0,"It that happens, just pull the plug.[SEP]if th..."
1,ENG-train-0001,A black dog running through water.\nA black do...,1.0,A black dog running through water.[SEP]A black...
2,ENG-train-0002,I've been searchingthe entire abbey for you.\n...,1.0,I've been searchingthe entire abbey for you.[S...
3,ENG-train-0003,If he is good looking and has a good personali...,1.0,If he is good looking and has a good personali...
4,ENG-train-0004,"She does not hate you, she is just annoyed wit...",1.0,"She does not hate you, she is just annoyed wit..."


In [9]:
#train test split

train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

train_dataset = str_dataset(train_df.reset_index(drop=True))
val_dataset = str_dataset(val_df.reset_index(drop=True))

str_datamodule = STR_DataModule(train_dataset=train_dataset, val_dataset= val_dataset, batch_size = 16)

/Users/lemarx/anaconda3/envs/str23/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
# print(train_dataset.__getitem__(0)[0].shape)
# print(train_dataset.__getitem__(1)[0].shape)

In [11]:
# test_input,test_att,_ = train_dataset.__getitem__(0)
# test_input
# with torch.no_grad():
#   model.eval()
#   output = model.deberta(test_input.unsqueeze(dim=0),test_att.unsqueeze(dim=0))

# print(output.hidden_states[-1][:, 0, :])

In [12]:
model = SentenceSimilarityModel()

trainer = pl.Trainer(max_epochs=25,callbacks=[EarlyStopping(monitor="val_loss", patience=3, mode="min")],accelerator='gpu')

# tuner = pl.tuner.Tuner(trainer)
# lr_finder = tuner.lr_find(model, datamodule = str_datamodule)

# # Pick point based on plot, or get suggestion
# new_lr = lr_finder.suggestion()

# # # update hparams of the model
# model.lr = new_lr

trainer.fit(model, datamodule=str_datamodule)

You are using a model of type deberta-v2 to instantiate a model of type deberta. This is not supported for all configurations of models and can yield errors.
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['deberta.encoder.layer.5.attention.self.v_bias', 'deberta.encoder.layer.1.attention.self.q_bias', 'deberta.encoder.layer.1.attention.self.in_proj.weight', 'deberta.encoder.layer.4.attention.self.in_proj.weight', 'deberta.encoder.layer.5.attention.self.pos_q_proj.bias', 'deberta.encoder.layer.4.attention.self.v_bias', 'pooler.dense.weight', 'deberta.encoder.layer.0.attention.self.q_bias', 'deberta.encoder.layer.0.attention.self.v_bias', 'deberta.encoder.layer.1.attention.self.pos_q_proj.weight', 'deberta.encoder.layer.4.attention.self.q_bias', 'deberta.encoder.layer.0.attention.self.pos_q_proj.weight', 'deberta.encoder.layer.1.attention.self.pos_proj.weight', 'deberta.encoder.laye

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/lemarx/anaconda3/envs/str23/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/lemarx/anaconda3/envs/str23/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'str_dataset' on <module '__main__' (built-in)>
/Users/lemarx/anaconda3/envs/str23/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
